# Translate Tables

## Introduction

This project is design to translate Chinese charactors in a table to English, so that the table can be handled by some English-based packages. The assumption here is that the meaning of the content is not essential and we only care about the structure of a phrase. 

In [1]:
from googletrans import Translator

translator = Translator()

In [2]:
translator.translate('안녕하세요.')

In [5]:
example = translator.translate('안녕하세요.')

In [9]:
example.text

'Good morning.'

In [11]:
translator.translate('科苑路').text

'Keyuan Road'

## Make A Practice Dataset

The data contains regulation information of company in Shanghai, downloaded from [datashang](http://www.datashanghai.gov.cn/). The made data contains 5% duplicated companies but each row has a unique index. 

In [12]:
import os
import pandas as pd

wd = '/Users/ewenwang/Documents/practice_data'
file = 'regulation.csv'

os.chdir(wd)
regulation = pd.read_csv(file)

In [13]:
regulation.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [53]:
regulation.tail()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
36667,36668,上海吉亿电机有限公司,782829293,上海市松江区九亭镇八佰伴路７７号,松江区九亭镇高科技园区洋河浜路180号3号楼,3/1/17 11:03,在检查“场（厂）内机动车辆使用情况(叉车，5110310117200801006 7)”时发...
36668,36669,上海鸿日物业发展有限公司,132285562,上海市浦东新区浦东大道２０００号１３楼Ｉ室,新华路2号,3/1/17 11:03,在检查“电梯使用情况(曳引式客梯，31103101052006040033)”时 发现以下问...
36669,36670,上海市金山区谭晓峰废旧物资回收站,NaN,NaN,NaN,3/1/17 11:03,现场查见2台叉车，企业反映1台已报检，1台新买报检中，要求企业提供相关证明。
36670,36671,上海市宝山区计量质量检测所,425050871,上海市友谊支路７５号,友谊支路75号,3/1/17 11:03,未发现问题。
36671,36672,上海电力环保设备总厂有限公司,133030473,上海市宝山区真陈路８８９号,上海市宝山区真陈路８８９号,3/1/17 12:03,在检查“特种设备使用单位现场安全监督检查安全管理情况”时发现以下问题： 1、未配 备


In [14]:
regulation.shape

(36672, 7)

In [55]:
seed = 2017
n_dup = round(regulation.shape[0]*.05)
duplication = regulation.sample(n=n_dup, replace=True, random_state=seed)

In [56]:
duplication.shape

(1834, 7)

In [57]:
duplication.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
16186,16187,上海嘉方市政建筑有限公司,631661753,上海市嘉定区安亭镇翔方公路２３７７号,嘉定,5/31/16 2:05,未发现问题。
8963,8964,上海尚原餐饮管理有限公司,692932609,上海市浦东新区苗圃路６３号１－３层,上海市浦东新区苗圃路６３号１－３层,11/20/15 10:11,未发现问题。
7727,7728,上海万辉汽车附件有限公司,630910922,上海市浦东新区万祥镇万灵路１３８号,南汇万祥工业区,9/24/15 9:09,未发现问题。
1224,1225,上海申得欧有限公司,607299571,上海市浦东新区庆达路２８８号,庆达路288号,3/12/14 3:03,未发现问题。 其他问题：叉车未张贴有效期内的检验合格标志
30208,30209,上海顺城物流有限公司,07483557X,上海市闵行区莲花南路２５８８号５幢二层Ｂ区,上海市闵行区莲花南路２５８８号５幢二层Ｂ区,12/5/16 11:12,在检查“场（厂）内机动车辆使用情况(叉车，5110310112201512003 7)”时发...


In [60]:
duplication['序号'] = range(regulation.shape[0]+1, regulation.shape[0]+1+duplication.shape[0])
duplication.index = range(regulation.shape[0]+1, regulation.shape[0]+1+duplication.shape[0])

In [61]:
duplication.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
36673,36673,上海嘉方市政建筑有限公司,631661753,上海市嘉定区安亭镇翔方公路２３７７号,嘉定,5/31/16 2:05,未发现问题。
36674,36674,上海尚原餐饮管理有限公司,692932609,上海市浦东新区苗圃路６３号１－３层,上海市浦东新区苗圃路６３号１－３层,11/20/15 10:11,未发现问题。
36675,36675,上海万辉汽车附件有限公司,630910922,上海市浦东新区万祥镇万灵路１３８号,南汇万祥工业区,9/24/15 9:09,未发现问题。
36676,36676,上海申得欧有限公司,607299571,上海市浦东新区庆达路２８８号,庆达路288号,3/12/14 3:03,未发现问题。 其他问题：叉车未张贴有效期内的检验合格标志
36677,36677,上海顺城物流有限公司,07483557X,上海市闵行区莲花南路２５８８号５幢二层Ｂ区,上海市闵行区莲花南路２５８８号５幢二层Ｂ区,12/5/16 11:12,在检查“场（厂）内机动车辆使用情况(叉车，5110310112201512003 7)”时发...


In [68]:
duplication.tail()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
38502,38502,上海沪菲电缆有限公司,774778246,上海市松江区中山街道荣乐东路１８号Ａ区,上海市金山区朱泾镇温河村新民1109号,5/31/16 2:05,在检查“场（厂）内机动车辆使用情况(叉车，5110310116201205002 2)”时发...
38503,38503,上海乐益物业有限公司,134501417,上海市普育东路１８５号１０２室,重庆北路318号,4/18/16 2:04,设备登记使用单位为上海捷艾尔物业管理有限公司。移动端登记为上海乐益物业有限公司。
38504,38504,上海凯顿酒店管理有限公司,776252678,上海市浦东唐镇创新中路６０１号８幢２０１室,上海市浦东唐镇创新中路６０１号８幢２０１室,11/2/15 3:11,未发现问题。
38505,38505,上海凡宏实业有限公司,630977326,上海市松江区洞泾镇洞舟路６１９号,上海市松江区洞泾镇洞舟路６１９号,7/8/16 12:07,未发现问题。
38506,38506,上海富达日用化工厂,134423527,上海市崇明县北沿公路１２１８号,上海市崇明县前进农场,12/27/13 10:12,未发现问题。


In [65]:
regulation_dup = pd.concat([regulation, duplication])
regulation_dup.shape

(38506, 7)

In [66]:
regulation_dup.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [67]:
regulation_dup.tail()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
38502,38502,上海沪菲电缆有限公司,774778246,上海市松江区中山街道荣乐东路１８号Ａ区,上海市金山区朱泾镇温河村新民1109号,5/31/16 2:05,在检查“场（厂）内机动车辆使用情况(叉车，5110310116201205002 2)”时发...
38503,38503,上海乐益物业有限公司,134501417,上海市普育东路１８５号１０２室,重庆北路318号,4/18/16 2:04,设备登记使用单位为上海捷艾尔物业管理有限公司。移动端登记为上海乐益物业有限公司。
38504,38504,上海凯顿酒店管理有限公司,776252678,上海市浦东唐镇创新中路６０１号８幢２０１室,上海市浦东唐镇创新中路６０１号８幢２０１室,11/2/15 3:11,未发现问题。
38505,38505,上海凡宏实业有限公司,630977326,上海市松江区洞泾镇洞舟路６１９号,上海市松江区洞泾镇洞舟路６１９号,7/8/16 12:07,未发现问题。
38506,38506,上海富达日用化工厂,134423527,上海市崇明县北沿公路１２１８号,上海市崇明县前进农场,12/27/13 10:12,未发现问题。


In [69]:
regulation_dup.to_excel('regulation_dup.xlsx', index = False)

## Practice

Task: Try to find out all indexes that belong to each company via its registered address and production address.

In [70]:
sample = regulation_dup.head(10)
sample.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [74]:
sample.columns = ['index', 'name', 'company_id', 'registered_add', 'production_add', 'date', 'content']

In [75]:
sample.head()

,index,name,company_id,registered_add,production_add,date,content
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [80]:
regulation_dup.columns = ['index', 'name', 'company_id', 'registered_add', 'production_add', 'date', 'content']

In [81]:
regulation_dup.head()

,index,name,company_id,registered_add,production_add,date,content
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [76]:
import recordlinkage

indexer = recordlinkage.FullIndex()

In [82]:
pairs = indexer.index(regulation_dup)
indexer = recordlinkage.BlockIndex(on='registered_add')
pairs = indexer.index(regulation_dup)

print (len(pairs))

56051


In [83]:
compare_cl = recordlinkage.Compare()

compare_cl.string('registered_add', 'registered_add', method='jarowinkler', threshold=0.85, label='registered_add')
compare_cl.string('production_add', 'production_add', method='jarowinkler', threshold=0.85, label='production_add')

features = compare_cl.compute(pairs, regulation_dup)

In [84]:
features.head(10)

registered_add  production_add
2     21530             1.0             1.0
3     23032             1.0             1.0
4     20566             1.0             1.0
      27518             1.0             1.0
      36223             1.0             1.0
20566 27518             1.0             1.0
      36223             1.0             1.0
27518 36223             1.0             1.0
5     21989             1.0             1.0
6     33683             1.0             1.0

In [100]:
# features.index.values
pairs.values[0:10]

array([(2, 21530), (3, 23032), (4, 20566), (4, 27518), (4, 36223),
       (20566, 27518), (20566, 36223), (27518, 36223), (5, 21989),
       (6, 33683)], dtype=object)

In [101]:
pairs.values[0]

(2, 21530)

In [106]:
print(pairs.values[0], 
      pairs.values[0][0], regulation_dup.values[pairs.values[0][0],1], 
      pairs.values[0][1], regulation_dup.values[pairs.values[0][1],1])

(2, 21530) 2 上海地矿物业管理有限公司 21530 上海地矿物业管理有限公司


In [113]:
def print_pairs(pairs, n=10):
    for i in range(n):
        print(pairs.values[i], 
              pairs.values[i][0], regulation_dup.values[pairs.values[i][0],1], 
              pairs.values[i][1], regulation_dup.values[pairs.values[i][1],1])
    return None

In [114]:
print_pairs(pairs, 10)

(2, 21530) 2 上海地矿物业管理有限公司 21530 上海地矿物业管理有限公司
(3, 23032) 3 上海立新五金厂 23032 上海立新五金厂
(4, 20566) 4 上海同祁置业有限公司 20566 上海同祁置业有限公司
(4, 27518) 4 上海同祁置业有限公司 27518 上海同祁置业有限公司
(4, 36223) 4 上海同祁置业有限公司 36223 上海同祁置业有限公司
(20566, 27518) 20566 上海同祁置业有限公司 27518 上海同祁置业有限公司
(20566, 36223) 20566 上海同祁置业有限公司 36223 上海同祁置业有限公司
(27518, 36223) 27518 上海同祁置业有限公司 36223 上海同祁置业有限公司
(5, 21989) 5 上海高地物业管理有限公司虹口分公司 21989 上海高地物业管理有限公司虹口分公司
(6, 33683) 6 上海康通电缆组件有限公司 33683 上海康通电缆组件有限公司


In [85]:
features.describe()

,registered_add,production_add
count,56051.0,56051.000000
mean,1.0,0.960143
std,0.0,0.195624
min,1.0,0.000000
25%,1.0,1.000000
50%,1.0,1.000000
75%,1.0,1.000000
max,1.0,1.000000


In [86]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0    53817
1.0     2234
dtype: int64

In [88]:
matches = features[features.sum(axis=1) > 0]

print(len(matches))
matches.head(10)

56051


registered_add  production_add
2     21530             1.0             1.0
3     23032             1.0             1.0
4     20566             1.0             1.0
      27518             1.0             1.0
      36223             1.0             1.0
20566 27518             1.0             1.0
      36223             1.0             1.0
27518 36223             1.0             1.0
5     21989             1.0             1.0
6     33683             1.0             1.0